In [2]:
import numpy as np, pandas as pd

import matplotlib.pyplot as plt
import plotly.express as px

import os, time
import urllib.request 

from biopandas.pdb import PandasPdb
import blosum as bl

from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

import Levenshtein
from Levenshtein import distance as levenshtein_distance

In [3]:
base = "VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK"

testCsvPath = "../novozymes-enzyme-stability-prediction-kaggle-2022/data/test.csv"
trainCsvPath = "../novozymes-enzyme-stability-prediction-kaggle-2022/data/train.csv"
pdbPath = "../novozymes-enzyme-stability-prediction-kaggle-2022/sample_data/wildtype_structure_prediction_af2.pdb"

testDF = pd.read_csv(testCsvPath)
trainDF = pd.read_csv(trainCsvPath)
pdbDF =  PandasPdb().read_pdb(pdbPath)

In [4]:
trainDF.values[:,1:]

array([['AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVGMIKDAGDDPDVTHGAEIQAFVRFASEDRLEGGEGVGVVTKPGLGVPVGEPAINPVPRRMIWEAVREVTERPLAVTIAIPGGEELAKKTLNPRLGILGGLSVLGTTGVVKPYSTSAFRMSVVQAVGVARANGLLEIAATTGGKSERFAQRLLPHLPEMAFIEMGDFVGDVLRAARKVGVEVVRVVGMIGKISKMADGKTMTHAAGGEVNLSLLLSLLKEAGASPKALKEAEGAATARRFLEIALEEGLELFFVNLVRLAQEKLQAYIGERPFVSVALTDFDEGRCLAAWPDREVYR',
        7.0, 'doi.org/10.1038/s41592-020-0801-4', 75.7],
       ['AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSRLQAQRRAQRVAWEDGDENVGQTVIPAQEEEGIEKPAEVHPTGKIGAKKLRKLEEKQARKAQREAEEAEREERKRLESQREAEWKKEEERLRLKEEQKEEEERKAQEEQARREHEEYLKLKEAFVVEEEGVSETMTEEQSHSFLTEFINYIKKSKVVLLEDLAFQMGLRTQDAINRIQDLLTEGTLTGVIDDRGKFIYITPEELAAVANFIRQRGRVSITELAQASNSLISWGQDLPAQAS',
        7.0, 'doi.org/10.1038/s41592-020-0801-4', 50.5],
       ['AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYATLGVAKNANGKDIKKAYYQLAKKYHPDTNKEDPDAGRKFQEVSEAYEVLSDEQKRREYDTYGQTAENIGRQGGGFPGGGAGGFGPEGFSQSWQFRSSIDPEELFRKIFGEGNFRTNSFDDFADSKFGFGQAQEMVMDLTFAQAARGVNKDVNVNVVDQCPKCAGTKCEPGTKPGRCQYCN

In [5]:
len(base)

221

In [6]:
len_list = [len(trainDF.values[i,1]) for i in range(len(trainDF))]

In [7]:
trainDF.insert(2, "sequence_len", len_list)

In [8]:
df1 = trainDF[trainDF.sequence_len == 221]
df2 = trainDF[trainDF.sequence_len == 220]
df3 = trainDF[trainDF.sequence_len == 222]

singleDF = pd.concat([df1,df2,df3])

In [9]:
def build_change_list(group_df):
    
    list_output = []
    group_size = len(group_df)
    group_values = group_df.values
    
    col = ['sequence_len','pH','data_source','tm']

    for i in range(group_size):
        data1 = group_values[i]
        line1 = data1[1] # protein sequence
        values1  = data1[2:]
        for j in range(group_size):
            data2 = group_values[j]
            line2 = data2[1]
            values2  = data2[2:]
            if i!=j:
                edits = Levenshtein.editops(line1, line2)
                if len(edits)==1:
                    list_output.append(tuple([line1,line2])+edits[0]+tuple(line1[edits[0][1]])+tuple(line2[edits[0][1]])+ tuple(values1) + tuple(values2))
                else:
                    list_output.append(tuple([line1,line2])+('replace', 0, 0, 'A', 'A') + tuple(values1) + tuple(values2))

    changes = pd.DataFrame(list_output,columns=['seq1','seq2','operation','position1','position2','change1','change2']+[c+'1' for c in col] + [c+'2' for c in col])
    changes.change2 = np.where(changes.operation=='delete','',changes.change2)
    
    return changes

In [10]:
changes = build_change_list(singleDF)

In [11]:
changes

seq1  \
0      ASLSQASSEGTTTCKAHDVCLLGPRPLPPSPPVRVSLYYESLCGAC...   
1      ASLSQASSEGTTTCKAHDVCLLGPRPLPPSPPVRVSLYYESLCGAC...   
2      ASLSQASSEGTTTCKAHDVCLLGPRPLPPSPPVRVSLYYESLCGAC...   
3      ASLSQASSEGTTTCKAHDVCLLGPRPLPPSPPVRVSLYYESLCGAC...   
4      ASLSQASSEGTTTCKAHDVCLLGPRPLPPSPPVRVSLYYESLCGAC...   
...                                                  ...   
21751  RFQSSVRTPASEPSAEKGVDEWLEAINELREEFSAKEYLPETSLAP...   
21752  RFQSSVRTPASEPSAEKGVDEWLEAINELREEFSAKEYLPETSLAP...   
21753  RFQSSVRTPASEPSAEKGVDEWLEAINELREEFSAKEYLPETSLAP...   
21754  RFQSSVRTPASEPSAEKGVDEWLEAINELREEFSAKEYLPETSLAP...   
21755  RFQSSVRTPASEPSAEKGVDEWLEAINELREEFSAKEYLPETSLAP...   

                                                    seq2 operation  position1  \
0      ETLAARGAKVIGTATSENGAQAISDYLGANGKGLMLNVTDPASIES...   replace          0   
1      FFIYLLRRQIRTVIQYQTVRYDILPLSPLSRNRLAQVKRKILVLDL...   replace          0   
2      KEEVPDNPPNEIYATAQQKLQDGNWRQAITQLEALDNRYPFGPYSQ...   replace          0   
3      KLGIISPAYFFLWPEAFLYRFQIWRPFTATFYFPVGPGTGFLYLVN...   replace          0   
4      KTKVYSRYGREIYVCAKAGGADPTANLSLRGMIERAKKDQVPSHVI...   replace          0   
...                                                  ...       ...        ...   
21751  MTEVLDLHGQDSDGGSEEMVLTPAELIEKLEQAWMNEKFAPELLES...   replace          0   
21752  MTKHIALLGGTFDPIHIGHLRMAIELRLAGFDEVRLIPNNVPPHRE...   replace          0   
21753  MTKQHANWSPYDNNGGTCVAIAGSDYCVIAADTRMSTGYSILSRDY...   replace          0   
21754  MTSEVIEDEKQFYSKAKTYWKQIPPTVDGMLGGYGHISNIDLNSSR...   replace          0   
21755  MTSSATSPTNGVDKNKNEEMVATPANCPYQLFNQEVVWNGKWIQTR...   replace          0   

       position2 change1 change2  sequence_len1  pH1  \
0              0       A       A            221  7.0   
1              0       A       A            221  7.0   
2              0       A       A            221  7.0   
3              0       A       A            221  7.0   
4              0       A       A            221  7.0   
...          ...     ...     ...            ...  ...   
21751          0       A       A            222  7.0   
21752          0       A       A            222  7.0   
21753          0       A       A            222  7.0   
21754          0       A       A            222  7.0   
21755          0       A       A            222  7.0   

                            data_source1   tm1  sequence_len2  pH2  \
0      doi.org/10.1038/s41592-020-0801-4  53.8            221  7.0   
1      doi.org/10.1038/s41592-020-0801-4  53.8            221  7.0   
2      doi.org/10.1038/s41592-020-0801-4  53.8            221  7.0   
3      doi.org/10.1038/s41592-020-0801-4  53.8            221  7.0   
4      doi.org/10.1038/s41592-020-0801-4  53.8            221  7.0   
...                                  ...   ...            ...  ...   
21751  doi.org/10.1038/s41592-020-0801-4  46.1            222  7.0   
21752  doi.org/10.1038/s41592-020-0801-4  46.1            222  7.0   
21753  doi.org/10.1038/s41592-020-0801-4  46.1            222  7.0   
21754  doi.org/10.1038/s41592-020-0801-4  46.1            222  7.0   
21755  doi.org/10.1038/s41592-020-0801-4  46.1            222  7.0   

                            data_source2   tm2  
0      doi.org/10.1038/s41592-020-0801-4  50.0  
1      doi.org/10.1038/s41592-020-0801-4  48.3  
2      doi.org/10.1038/s41592-020-0801-4  47.9  
3      doi.org/10.1038/s41592-020-0801-4  50.7  
4      doi.org/10.1038/s41592-020-0801-4  37.7  
...                                  ...   ...  
21751  doi.org/10.1038/s41592-020-0801-4  49.9  
21752  doi.org/10.1038/s41592-020-0801-4  48.1  
21753  doi.org/10.1038/s41592-020-0801-4  66.9  
21754  doi.org/10.1038/s41592-020-0801-4  46.0  
21755  doi.org/10.1038/s41592-020-0801-4  45.7  

[21756 rows x 15 columns]

In [12]:
# df_clean = changes[(changes.data_source1 == changes.data_source2)&(changes.pH1 == changes.pH2)]
# df_clean = df_clean[(df_clean.pH1>=6) & (df_clean.pH1<=8)]
df_clean = changes[changes.position1 != 0]

df_clean['target'] = df_clean['tm2'] - df_clean['tm1'] 

print(len(df_clean))
display(pd.crosstab(df_clean.change1,df_clean.change2).style.background_gradient(axis=None, cmap="YlGnBu"))

C:\Users\XTM233\AppData\Local\Temp\ipykernel_2208\3815420638.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['target'] = df_clean['tm2'] - df_clean['tm1']


44


In [13]:
df_clean

seq1  \
16132  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   
16133  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   
16134  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   
16135  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   
16136  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   
16137  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   
16138  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   
16279  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   
16280  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   
16281  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   
16282  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   
16283  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   
16284  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   
16285  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   
16426  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   
16427  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   
16431  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   
16432  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   
16573  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   
16574  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   
16578  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   
16579  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   
16720  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   
16721  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   
16725  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   
16726  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   
16867  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   
16868  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   
16872  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   
16873  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   
17014  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   
17015  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   
17016  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   
17017  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   
17018  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   
17019  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   
17020  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   
17161  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   
17162  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   
17163  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   
17164  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   
17165  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   
17166  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   
17167  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   

                                                    seq2 operation  position1  \
16132  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   replace        166   
16133  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   replace        166   
16134  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   replace        166   
16135  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   replace        166   
16136  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   replace        166   
16137  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   replace        166   
16138  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   replace        166   
16279  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   replace        166   
16280  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   replace        166   
16281  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   replace        166   
16282  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   replace        166   
16283  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   replace        166   
16284  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   replace        166   
16285  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   replace        166   
16426  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI...   replace        166   
16427  MLSRAVCGTSRQLAPVLGYLGSRQKHSLPDLPYDYGALEPHINAQI..

In [14]:
avg_target = df_clean.groupby(['change1','change2']).target.mean()
display(avg_target.unstack().fillna(0).style.background_gradient(axis=None, cmap="RdYlBu").format('{:.2f}'))

In [15]:
# compared to test data

def change_list(wild, mutation_list):

    list_output = []

    for mutation in mutation_list:
        edits = Levenshtein.editops(wild, mutation)
        if len(edits):
            list_output.append(edits[0]+tuple(mutation[edits[0][1]])+tuple(base[edits[0][1]]))
        else:
            list_output.append(('replace', 0, 0, 'A', 'A'))

    changes = pd.DataFrame(list_output,columns=['operation','position1','position2','change1','change2'])
    changes.change2 = np.where(changes.operation=='delete','',changes.change2)
    
    return changes

changes = change_list(base, testDF.protein_sequence.to_list())
changes.operation.value_counts()

replace    2336
delete       77
Name: operation, dtype: int64

In [16]:
changes.position1.hist()

<AxesSubplot: >

In [17]:
pd.crosstab(changes.change1,changes.change2).style.background_gradient(axis=None, cmap="YlGnBu")

In [18]:
testDF.head

<bound method NDFrame.head of       seq_id                                   protein_sequence  pH  \
0      31390  VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVD...   8   
1      31391  VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVD...   8   
2      31392  VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDC...   8   
3      31393  VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVD...   8   
4      31394  VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVD...   8   
...      ...                                                ...  ..   
2408   33798  VPVNPEPDATSVENVILKTGSGDSQSDPIKADLEVKGQSALPFDVD...   8   
2409   33799  VPVNPEPDATSVENVLLKTGSGDSQSDPIKADLEVKGQSALPFDVD...   8   
2410   33800  VPVNPEPDATSVENVNLKTGSGDSQSDPIKADLEVKGQSALPFDVD...   8   
2411   33801  VPVNPEPDATSVENVPLKTGSGDSQSDPIKADLEVKGQSALPFDVD...   8   
2412   33802  VPVNPEPDATSVENVWLKTGSGDSQSDPIKADLEVKGQSALPFDVD...   8   

     data_source  
0      Novozymes  
1      Novozymes  
2      Novozymes  
3      Novozymes  
4      Novozymes  
...

In [19]:
dict_enzyme = {
    'ALA':'A',
    'ARG':'R',
    'ASN':'N',
    'ASP':'D',
    'CYS':'C',
    'GLU':'E',
    'GLN':'Q',
    'GLY':'G',
    'HIS':'H',
    'ILE':'I',
    'LEU':'L',
    'LYS':'K',
    'MET':'M',
    'PHE':'F',
    'PRO':'P',
    'SER':'S',
    'THR':'T',
    'TRP':'W',
    'TYR':'Y',
    'VAL':'V'
}

atom_df = pdbDF.df['ATOM']
atom_df['residue_letter'] = atom_df.residue_name.map(dict_enzyme)

In [20]:
testDF['modif'] = changes.position1

map_number_to_b = atom_df.groupby('residue_number').b_factor.mean()
testDF['b_factor'] = (testDF.modif+1).map(map_number_to_b).fillna(0)

atom_df['distance'] = np.sqrt(np.square(atom_df.x_coord) + np.square(atom_df.y_coord) + np.square(atom_df.z_coord))
map_number_to_distance = atom_df.groupby('residue_number').distance.mean()
testDF['distance'] = (testDF.modif+1).map(map_number_to_distance).fillna(0)

plt.scatter(map_number_to_b.index, map_number_to_b.values)

In [21]:
# try to improve b-factor prediction by composition of amino acid - not really working
#remove outliers ?

atom_remove_low = atom_df[atom_df.b_factor>80].copy()

X = atom_remove_low.groupby(['residue_number','element_symbol']).size().unstack(fill_value=0)
y = atom_remove_low.groupby(['residue_number']).b_factor.mean()

reg = LinearRegression().fit(X, y)

X.index = atom_remove_low.groupby('residue_number').residue_letter.first()

value_by_letter = pd.DataFrame({'letter':X.index, 'pred':reg.predict(X)})
map_b_by_letter = value_by_letter.groupby('letter').mean()

changes['Delta1'] = changes.change1.map(map_b_by_letter['pred'])
changes['Delta2'] = changes.change2.map(map_b_by_letter['pred'])
changes['Delta'] = (changes['Delta2'] - changes['Delta1']).fillna(0)